# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from Keyring import Google

# Configure gmaps
gmaps.configure(api_key=Google)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import data from Part 1
Cities = "Output/CityData.csv"


# Bring it into a DataFrame
CityData = pd.read_csv(Cities)

CityData

,City,Lat,Lon,Temp,HUM,Cloud,Wind
0,Buchanan,5.8808,-10.0467,78.80,94,40,12.21
1,Busselton,-33.6500,115.3333,55.00,96,6,3.11
2,Ushuaia,-54.8000,-68.3000,41.00,70,20,4.61
3,Saint-Leu,-21.1500,55.2833,75.20,69,0,8.05
4,Derzhavinsk,51.1000,66.3167,49.91,47,59,10.16
...,...,...,...,...,...,...,...
581,Huarmey,-10.0681,-78.1522,71.64,76,67,10.02
582,San Jose,37.3394,-121.8950,75.00,49,20,18.41
583,Toumodi,6.5613,-5.0158,84.00,78,20,4.61
584,Grand Gaube,-20.0064,57.6608,75.00,88,11,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Set up the data
locations = CityData[["Lat", "Lon"]].astype(float)
PctHum = CityData["HUM"].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure(center = (0,0),zoom_level = 1.5)

humidity = gmaps.heatmap_layer(locations, weights=PctHum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(humidity)

fig

Figure(layout=FigureLayout(height='420px'))

* Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Start searching for the perfect vacation spot
Perfect = CityData.copy()
Perfect = Perfect.dropna()

# Not a big fan of humidity
Perfect = CityData.loc[CityData['HUM'] < 30]
Perfect = Perfect.loc[Perfect['HUM'] > 20]

# Not too hot, not too cold, juuuust right.
Perfect = Perfect.loc[Perfect['Temp'] <85 ]
Perfect = Perfect.loc[Perfect['Temp'] > 70 ]

# What are my options?
Perfect

,City,Lat,Lon,Temp,HUM,Cloud,Wind
22,Tukrah,32.5341,20.5791,83.79,23,1,5.93
46,Morehead,37.2711,-87.1764,77.00,29,1,5.75
58,Huntsville,34.7304,-86.5861,79.00,24,1,3.44
93,Luderitz,-26.6481,15.1594,74.64,28,0,13.60
220,Rapid Valley,44.0625,-103.1463,78.80,21,1,8.05
327,Balkanabat,39.5108,54.3671,74.39,27,96,10.83
370,Evanston,42.0411,-87.6901,82.40,28,20,31.07
570,Mamu Kanjan,30.8333,72.8000,83.16,27,23,6.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = Perfect.copy()

# Set up some empty boxes
hotel_df['Hotel Name'] = ""
hotel_df['Country'] = ""

# Make sure I did it right
hotel_df


,City,Lat,Lon,Temp,HUM,Cloud,Wind,Hotel Name,Country
22,Tukrah,32.5341,20.5791,83.79,23,1,5.93,,
46,Morehead,37.2711,-87.1764,77.00,29,1,5.75,,
58,Huntsville,34.7304,-86.5861,79.00,24,1,3.44,,
93,Luderitz,-26.6481,15.1594,74.64,28,0,13.60,,
220,Rapid Valley,44.0625,-103.1463,78.80,21,1,8.05,,
327,Balkanabat,39.5108,54.3671,74.39,27,96,10.83,,
370,Evanston,42.0411,-87.6901,82.40,28,20,31.07,,
570,Mamu Kanjan,30.8333,72.8000,83.16,27,23,6.42,,


In [6]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # pull in variables and whatnot
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    coord = str(row['Lat'])+ "," +str(row['Lon'])
    rad = 5000
    hotel = 'lodging'
    key = Google
    city = row['City']
    
    Parameters = {
    "location": coord,  
    "radius": rad,
    "type": hotel,
    "key": key,
    }
  
    # assemble url and make API request
    print('Retrieving Results for ' + city)
    response = requests.get(base_url, params=Parameters).json()
    
    
    # extract results
    results = response['results']
  
    try:
               
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    
    except (KeyError, IndexError):
    
        print("Missing field/result... skipping.")

Retrieving Results for Tukrah
Retrieving Results for Morehead
Retrieving Results for Huntsville
Retrieving Results for Luderitz
Retrieving Results for Rapid Valley
Retrieving Results for Balkanabat
Retrieving Results for Evanston
Retrieving Results for Mamu Kanjan


In [7]:
# Since I didn't pull in Country on the original dataframe, it is being added here.  
for index, row in hotel_df.iterrows():

    # pull in variables and whatnot
    URL = "http://api.geonames.org/findNearbyPlaceNameJSON?lat=" + str(row["Lat"]) + "&lng=" + str(row["Lon"]) + "&username=gbfreed"
    city = row['City']
  

    # assemble url and make API request
    print('Retrieving Results for ' + city)
    response = requests.get(URL).json()
    
    try:
               
        hotel_df.loc[index, 'Country'] = response['geonames'][0]['countryName']

    
    except (KeyError, IndexError):
    
        print("Missing field/result... skipping.")


Retrieving Results for Tukrah
Retrieving Results for Morehead
Retrieving Results for Huntsville
Retrieving Results for Luderitz
Retrieving Results for Rapid Valley
Retrieving Results for Balkanabat
Retrieving Results for Evanston
Retrieving Results for Mamu Kanjan


In [8]:
hotel_df

,City,Lat,Lon,Temp,HUM,Cloud,Wind,Hotel Name,Country
22,Tukrah,32.5341,20.5791,83.79,23,1,5.93,Lokta restaurant,Libya
46,Morehead,37.2711,-87.1764,77.00,29,1,5.75,CCI Express Inn,United States
58,Huntsville,34.7304,-86.5861,79.00,24,1,3.44,Embassy Suites by Hilton Huntsville Hotel & Spa,United States
93,Luderitz,-26.6481,15.1594,74.64,28,0,13.60,LÜDERITZ NEST HOTEL,Namibia
220,Rapid Valley,44.0625,-103.1463,78.80,21,1,8.05,Fairfield Inn & Suites by Marriott Rapid City,United States
327,Balkanabat,39.5108,54.3671,74.39,27,96,10.83,Nebitchi Hotel,Turkmenistan
370,Evanston,42.0411,-87.6901,82.40,28,20,31.07,Hilton Garden Inn Chicago North Shore/Evanston,United States
570,Mamu Kanjan,30.8333,72.8000,83.16,27,23,6.42,Malik Habib & sons House,Pakistan


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
HotelLoc = hotel_df[["Lat", "Lon"]]


In [10]:
# Add marker layer ontop of heat map

# Create a humidity Heatmap layer
locations = CityData[["Lat", "Lon"]].astype(float)
PctHum = CityData["HUM"].astype(float)
fig = gmaps.figure(center = (0,0),zoom_level = 1.8)

humidity = gmaps.heatmap_layer(locations, weights=PctHum, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(humidity)


# Create a hotel layer
hotels = gmaps.symbol_layer(HotelLoc, hover_text='', fill_color=None, 
                               fill_opacity=1.0, stroke_color=None, stroke_opacity=1.0, scale=3, info_box_content=hotel_info, display_info_box=None)

fig.add_layer(hotels)

# Display figure

fig




Figure(layout=FigureLayout(height='420px'))